In [1]:
from collections.abc import Iterable, Iterator

## 可迭代对象 Iterable VS 迭代器 Iterator

In [2]:
class Color(object):

    def __init__(self):
        self.colors = ['red', 'white', 'black', 'green']

    def __iter__(self):
        pass

Color_obj_1 = Color()
print(isinstance(Color_obj_1, Iterable)) # 验证是否 可迭代对象
print(isinstance(Color_obj_1, Iterator)) # 验证是否 迭代器

True
False


1. 定义了__iter__方法的类, 是**可迭代对象 Iterable**。

In [3]:
class Color(object):

    def __init__(self):
        self.colors = ['red', 'white', 'black', 'green']

    # 仅仅是实现了__iter__ 方法,在方法内部什么都不做
    def __iter__(self):
        pass

    def __next__(self):
        pass

Color_obj_2 = Color()
print(isinstance(Color_obj_2, Iterable)) # 验证是否 可迭代对象
print(isinstance(Color_obj_2, Iterator)) # 验证是否 迭代器

True
True


2. 定义了__iter__方法 和__next__方法的类, 是**迭代器 Iterator**。

3. 不管是 可迭代对象还是迭代器，它们的__iter__方法通常都需要返回一个**迭代器 Iterator**。  
所以一个**迭代器 Iterator**的__iter__方法可以返回自身self。

iter 内置函数的作用 : iterable ----> iterator, 给一个 可迭代对象, 调用该 可迭代对象的 __iter__()方法, 返回一个迭代器

next 内置函数的作用 : iterator ----> element in order/stopIteration, 给一个 迭代器, 返回其中的下一个元素/StopIteration

总结1：在代码逻辑上，先有 可迭代对象 Iterable, 再有 迭代器 Iterator。因为 可迭代对象 Iterable 只需要实现__iter__方法，而 迭代器 Iterator 需要同时实现__iter__方法和__next__方法。

## 迭代器内的__iter__方法是否返回self, 决定了其singularity性质

In [7]:
class Color_sing(object):

    def __init__(self):
        self.index = -1
        self.colors = ['red', 'white', 'black', 'green']

    def __iter__(self):
        return self

    def __next__(self):
        # 每调用一次 __next__()方法, self.index + 1
        self.index += 1
        # 界定遍历结束的边界
        if self.index >= len(self.colors):
            raise StopIteration
        # 返回每次遍历应该返回的元素
        return self.colors[self.index]

color = Color_sing() # Color_sing类，是个迭代器，也是个可迭代对象
color_iter1 = iter(color) # 调用 __iter__ 方法，返回 迭代器1
color_iter2 = iter(color) # 调用 __iter__ 方法，返回 迭代器2
print(color_iter1 is color_iter2)
print(color_iter1 is color)
print(next(color_iter1))
print(next(color_iter2))

True
True
red
white


1. 具备singularity性质的迭代器 Iterator, 在内存中只有一个实例（即使有多个变量名，都引用到这同一个实例）。__next__调用实际都是调用同一个迭代器实例。  
例子有 file Iterator: 在同一时间环境，只希望这个file读取一遍

In [ ]:
class ListIterator:
    def __init__(self, lst):
        self.lst = lst
        self.index = -1
    
    def __iter__(self):
        return self
    
    def __next__(self):
        self.index += 1
        if self.index >= len(self.lst):
            raise StopIteration
        return self.lst[self.index]


class Color_nosing(object):

    def __init__(self):
        self.colors = ['red', 'white', 'black', 'green']

    def __iter__(self):
        return ListIterator(self.colors)

    def __next__(self):
        pass

color = Color_nosing() # Color_sing类，是个迭代器，也是个可迭代对象
color_iter1 = iter(color) # 调用 Color_nosing 类 的 __iter__ 方法，返回 迭代器1
color_iter2 = iter(color) # 调用 Color_nosing 类 的 __iter__ 方法，返回 迭代器2
print(color_iter1 is color_iter2)
print(color_iter1 is color)
print(type(color_iter1), type(color_iter2), type(color))
print(next(color_iter1)) # 调用的是 ListIterator 类 的 __next__方法
print(next(color_iter2)) # 调用的是 ListIterator 类 的 __next__方法
print(next(color)) # 调用的是 Color_nosing 类 的 __next__方法

False
False
<class '__main__.ListIterator'> <class '__main__.ListIterator'> <class '__main__.Color_nosing'>
red
red
None


2. 具备nonsingularity性质的迭代器 Iterator, 依靠__iter__方法多次调用，在内存中生成多个非自身的迭代器实例。  
例子有 List Iterator: 同一个 Python List Object，希望在同一时间环境，这个list可以多次读取

总结2：在实用业务抽象里，先实现具备singularity性质的迭代器（原子迭代器），这个原子迭代器的__iter__方法返回自身，从而保证它的原子性质：调用__iter__创造的迭代器只有它自己。迭代器同时也是可迭代对象，故若需要一个原子的可迭代对象，该原子迭代器就满足条件；若需要一个非原子的可迭代对象，多封装一层Iterable class即可。

总结3：如果普遍遵循这样的开发规范，可以发现：迭代器普遍是只可遍历一遍的，而封装一遍过后的可迭代对象，普遍是可以同时多次遍历的。

## for 循环 工作原理
for element_obj in Iterable 中, 首先使用 iter on Iterable 得到 Iterator, 然后使用 next on Iterator 不断得到 element,
最后捕捉 stopIteration